<a href="https://colab.research.google.com/github/PaweekornS/underwater_sonar/blob/main/synthesis_procedural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
!pip install Pillow

In [30]:
import os
import random
from PIL import Image
import numpy as np

# --- 0. ติดตั้ง NumPy (รันแค่ครั้งเดียวใน Colab) ---
try:
    import numpy as np
except ImportError:
    print("NumPy is not installed. Installing now...")
    !pip install numpy
    import numpy as np

# --- 1. Define Paths (ใช้ SeniorProj) ---
BACKGROUND_DIR = '/content/drive/MyDrive/SeniorProj/backgrounds'
OBJECT_DIR = '/content/drive/MyDrive/SeniorProj/objects'
OUTPUT_DIR = '/content/drive/MyDrive/SeniorProj/output'

# --- 2. Parameters Setup ---
NUM_OBJECTS_PER_BG = 1
MIN_SCALE_FACTOR = 0.05
MAX_SCALE_FACTOR = 0.15

# ค่าสีที่ถือว่าเป็น Object และ Shadow
OBJECT_COLOR = 255
SHADOW_COLOR = 0

# ช่วงความคลาดเคลื่อนที่ยอมรับได้สำหรับสีขาว/ดำ (Object/Shadow)
ACTIVE_COLOR_TOLERANCE = 5

# >>>>>> ปรับค่าความเข้ม (Strength) ตรงนี้ <<<<<<
# ค่าเดิม: LIGHTEN_STRENGTH = 0.25, DARKEN_STRENGTH = 0.35
LIGHTEN_STRENGTH = 0.5   # เพิ่มความสว่างของ Object ขึ้น 2 เท่า (จาก 0.25)
DARKEN_STRENGTH = 0.7    # เพิ่มความมืดของ Shadow ขึ้น 2 เท่า (จาก 0.35)


# --- 3. Core Function ---

def process_images_colorize_and_blend_increased_strength():
    print("--- Starting Colorize and Blend Process (Increased Strength) ---")

    # Mount Drive
    from google.colab import drive
    try:
        drive.mount('/content/drive', force_remount=True)
    except:
        print("!! Could not mount Google Drive. Please check connection.")
        return

    # สร้างโฟลเดอร์ Output
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # ดึงรายชื่อไฟล์
    try:
        background_files = [f for f in os.listdir(BACKGROUND_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        object_files = [f for f in os.listdir(OBJECT_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    except FileNotFoundError as e:
        print(f"**ERROR:** Path check failed. Please verify the directory paths: {e}")
        return

    # โหลด Object Masks
    objects = []
    for obj_file in object_files:
        try:
            obj_mask = Image.open(os.path.join(OBJECT_DIR, obj_file)).convert("L")
            objects.append(obj_mask)
        except Exception as e:
            print(f"!! Error loading Object Mask {obj_file}: {e}")

    if not objects:
        print("**ERROR:** Could not load any Object Masks.")
        return

    for bg_file in background_files:
        try:
            background_path = os.path.join(BACKGROUND_DIR, bg_file)
            composite_pil = Image.open(background_path).convert("RGB")
            bg_np = np.array(composite_pil, dtype=np.float32)
            bg_width, bg_height, _ = bg_np.shape

            # ใช้ composite_np สำหรับการประมวลผล Math Blending
            composite_np = bg_np.copy()

            for i in range(NUM_OBJECTS_PER_BG):
                random_object_mask = random.choice(objects)

                # สุ่มขนาดและหมุน
                scale_factor = random.uniform(MIN_SCALE_FACTOR, MAX_SCALE_FACTOR)
                new_obj_width = int(bg_width * scale_factor)

                w_percent = (new_obj_width / float(random_object_mask.size[0]))
                new_obj_height = int((float(random_object_mask.size[1]) * float(w_percent)))

                if new_obj_width == 0 or new_obj_height == 0: continue

                resized_mask = random_object_mask.resize((new_obj_width, new_obj_height), Image.Resampling.NEAREST)

                fill_color = 128
                rotated_mask = resized_mask.rotate(
                    random.randint(0, 359),
                    expand=True,
                    resample=Image.Resampling.NEAREST,
                    fillcolor=fill_color
                )

                mask_np = np.array(rotated_mask, dtype=np.float32)
                obj_h, obj_w = mask_np.shape

                # Random Position (x, y)
                max_x = bg_width - obj_w
                max_y = bg_height - obj_h

                if max_x <= 0 or max_y <= 0: continue

                random_x = random.randint(0, max_x)
                random_y = random.randint(0, max_y)

                # --- 1. สร้าง Active Mask ---

                # A. หาพิกเซลที่อยู่ในช่วงของ Object/Shadow
                is_object = np.logical_and(mask_np >= OBJECT_COLOR - ACTIVE_COLOR_TOLERANCE, mask_np <= OBJECT_COLOR + ACTIVE_COLOR_TOLERANCE)
                is_shadow = np.logical_and(mask_np >= SHADOW_COLOR - ACTIVE_COLOR_TOLERANCE, mask_np <= SHADOW_COLOR + ACTIVE_COLOR_TOLERANCE)
                is_active = np.logical_or(is_object, is_shadow)

                # D. สร้าง Alpha Channel (โปร่งใส 0/ทึบ 255)
                alpha_np = np.where(is_active, 255, 0).astype(np.uint8)
                alpha_channel_pil = Image.fromarray(alpha_np, mode='L')

                # --- 2. Math Blending บน Background RGB โดยตรง (Colorize) ---

                # Crop พื้นที่ Background มาเพื่อทำการ Blending
                y1, y2 = random_y, random_y + obj_h
                x1, x2 = random_x, random_x + obj_w
                bg_area_np = composite_np[y1:y2, x1:x2, :].copy()

                blended_area_np = bg_area_np.copy()

                for c in range(3): # Loop R, G, B channels
                    current_channel = bg_area_np[:, :, c]

                    # 1. Calculate Lighten (Object)
                    # out = original + (255 - original) * strength
                    lighten_value = current_channel + (255.0 - current_channel) * LIGHTEN_STRENGTH

                    # 2. Calculate Darken (Shadow)
                    # out = original * (1.0 - strength)
                    darken_value = current_channel * (1.0 - DARKEN_STRENGTH)

                    blended_channel = current_channel.copy()

                    # Apply Lighten for White areas (Object)
                    blended_channel = np.where(is_object, lighten_value, blended_channel)

                    # Apply Darken for Black areas (Shadow)
                    blended_channel = np.where(is_shadow, darken_value, blended_channel)

                    blended_area_np[:, :, c] = blended_channel

                # --- 3. แปะผลลัพธ์ Blend กลับบน Background (ใช้ PIL Paste) ---

                # แปลงผลลัพธ์การ Blend กลับเป็น PIL Image
                blended_object_rgb_pil = Image.fromarray(np.uint8(np.clip(blended_area_np, 0, 255)), mode='RGB')

                # สร้างภาพ RGBA เพื่อให้ paste รู้ว่าควรทำอะไรกับ Alpha Channel
                blended_object_rgba_pil = Image.merge('RGBA', blended_object_rgb_pil.split() + (alpha_channel_pil,))

                # แปะภาพ RGBA ลงบน Background RGB
                composite_pil.paste(blended_object_rgba_pil, (random_x, random_y), mask=blended_object_rgba_pil)

            # บันทึกภาพผลลัพธ์
            output_path = os.path.join(OUTPUT_DIR, f"composite_{bg_file}")
            composite_pil.save(output_path, quality=95)

            print(f"Saved: {output_path}")

        except Exception as e:
            print(f"!! ERROR processing Background {bg_file}: {e}")

print("\nProcessing complete!")

# --- 4. Colab Execution ---

# Run the main function
process_images_colorize_and_blend_increased_strength()
print("-" * 50)
print("Processing finished! Check your output folder in Google Drive. ")


Processing complete!
--- Starting Colorize and Blend Process (Increased Strength) ---
Mounted at /content/drive


/tmp/ipython-input-1283982247.py:132: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  alpha_channel_pil = Image.fromarray(alpha_np, mode='L')
/tmp/ipython-input-1283982247.py:167: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  blended_object_rgb_pil = Image.fromarray(np.uint8(np.clip(blended_area_np, 0, 255)), mode='RGB')


Saved: /content/drive/MyDrive/SeniorProj/output/composite_2021_11_19_exp2_0617_bmp.rf.e18cfb57651fe5b30af3eab6025d9774.jpg
Saved: /content/drive/MyDrive/SeniorProj/output/composite_2021_11_19_exp2_0614_bmp.rf.6b85f1f82961a90d8ab0b739334884d5.jpg
Saved: /content/drive/MyDrive/SeniorProj/output/composite_2021_11_19_exp2_0613_bmp.rf.983569995238b90abaa433dd3cfc98dc.jpg
Saved: /content/drive/MyDrive/SeniorProj/output/composite_2021_11_19_exp2_0612_bmp.rf.732d31dba8a8e02b5704c29f167b993b.jpg
Saved: /content/drive/MyDrive/SeniorProj/output/composite_2021_11_19_exp2_0611_bmp.rf.f429a39c3ab1662368f0e9455c510c37.jpg
Saved: /content/drive/MyDrive/SeniorProj/output/composite_2021_11_19_exp2_0600_bmp.rf.92babc41130e144ba149d0f69137bb48.jpg
Saved: /content/drive/MyDrive/SeniorProj/output/composite_2021_11_19_exp2_0609_bmp.rf.06ec5861f9599b09ab35d4fccc627ca5.jpg
Saved: /content/drive/MyDrive/SeniorProj/output/composite_2021_11_19_exp2_0599_bmp.rf.e78482a56296252b2856ab32e23328f7.jpg
Saved: /content/